In [6]:
from nltk import TweetTokenizer, ngrams
from nltk.tokenize import sent_tokenize, word_tokenize, ToktokTokenizer
from pandas import read_csv
import os
import time
from pprint import pprint
from joblib import Parallel, delayed

In [ ]:
def tokenize_phrase(phrase):
    tknzr = TweetTokenizer()
    return tknzr.tokenize(phrase)


def preprocess_corpus(corpus):
    tab_corpus = []
    for phrase in corpus
    


## Word-level N-Grams

In [ ]:
def spellchecker_1(data, gold_text, noizy_text, tokenized=False):
    if not tokenized:
        g_tokenized_text = word_tokenize(gold_text)
        n_tokenized_text = word_tokenize(noizy_text)
    else:
        g_tokenized_text = gold_text
        n_tokenized_text = noizy_text
        
    print(len(g_tokenized_text), len(n_tokenized_text))
        
    tokenized_text = zip(g_tokenized_text, n_tokenized_text)
    ngram_text = list(tokenized_text)#list(ngrams(tokenized_text, 1, pad_right=True, right_pad_symbol="p"))
    correct_data = []
    glossary = list(set(g_tokenized_text))
    
    for phrase in data:
        tokenized_phrase = ngrams(word_tokenize(phrase), 1)
        correct_phrase = []
        for token in tokenized_phrase:
            token = token[0]
            print(token)
            
            #teraz dla tych dwoch tokenow musze znalexc token maksymalizujacy prawdopodobienstwo p = p(t2| t1, c) * p(c| t1)
            #p_1 = p(t2| t1, c), mialo byc t1c, zapisano t1t2
            #p_2 = p(c| t1), zaobserwowano t1, jakie jest prawdopodobienstwo ze nastepnie bedzie c
            #albo p=p(t1,t2|c) * p(c)
            p_max = 0
            c_max = ""
            for c in glossary:
                sub_grams = [gram for gram in ngram_text if gram[0] == c]
                if sub_grams:
#                     print(sub_grams)
                    p_1 = len([gram for gram in sub_grams if gram[1] == token]) / len(sub_grams)
                    p_2 = len(sub_grams)
                    if p_1 * p_2 > p_max:
                        p_max = p_1 * p_2
                        c_max = c
            correct_phrase.append(c_max)
            
        correct_data.append(" ".join(correct_phrase))
        
    return correct_data

In [ ]:
def get_prob_table(c, ngram_text, token1, token2):
    sub_grams = [gram for gram in ngram_text if gram[1][0] == c]
    if sub_grams:
        p_1 = len([gram for gram in sub_grams if gram[0][1] == token1 and gram[1][1] == token2]) / len(sub_grams)
        p_2 = len(sub_grams)
#                     print(p_1 * p_2)
#                     print(sub_grams)
        if p_1 * p_2 > 0:
            return p_1 * p_2, c
    return None

In [ ]:
 #teraz dla tych dwoch tokenow musze znalexc token maksymalizujacy prawdopodobienstwo p = p(t2| t1, c) * p(c| t1)
#p_1 = p(t2| t1, c), mialo byc t1c, zapisano t1t2
#p_2 = p(c| t1), zaobserwowano t1, jakie jest prawdopodobienstwo ze nastepnie bedzie c
#albo p=p(t1,t2|c) * p(c)

def get_corrected_sentence(sentence, ngram_text, glossary, tokenizer):
    tokenized_phrase = list(ngrams(tokenizer.tokenize(sentence), 2))
    print(list(tokenized_phrase))
    correct_phrase = []
    dist_tab = set()
    for token1, token2 in tokenized_phrase:
        dist_tab = Parallel(n_jobs=-1)(delayed(get_prob_table)(c, ngram_text, token1, token2) for c in glossary)
        dist_tab = list(filter(lambda x: x is not None, dist_tab))
        print(dist_tab, len(dist_tab))
        if dist_tab:
            a = sorted(dist_tab, reverse=True)
            correct_phrase.append(a[0][1])
    return " ".join(correct_phrase)
#detokenized

In [ ]:
#assumptions:
#data is a list, data is given in phrases form - each element is a phrase
#corpus is a dict, corpus is given in golden_phrases : set_of_miss_spelled_phrases form
#p_corpus is a dict, is given in tokenized_golden_token: set_of_empirical_spelled_tokenized_phrases

#X - word in the window, 
#Y - observed word in the pre-window
#Z - observed word in the window
#P = (X|YZ)
# interpretation: observing the string, what expression most likely ends it


def spellchecker(data, ngram_text, n):   
    corrected_data = []
    toktok = ToktokTokenizer()
    
    for sentence in data:
        corrected_data.append(get_corrected_sentence(sentence, ngram_text, glossary, toktok))        
    
    return corrected_data

In [ ]:
def create_ngrams_from_text(df_duo, n):
    toktok = ToktokTokenizer()
    valid = list(df_duo["valid"].values)
    misspelled = list(df_duo["misspelled"].values)
    gold_text = " ".join(valid)
    noizy_text = " ".join(misspelled)
    
    g_tokenized_text = toktok.tokenize(gold_text)
    n_tokenized_text = toktok.tokenize(noizy_text)
        
    print(len(g_tokenized_text), len(n_tokenized_text))
    glossary = list(set(g_tokenized_text))
        
    tokenized_text = zip(g_tokenized_text, n_tokenized_text)
    ngram_text = list(ngrams(tokenized_text, n))
    
    return ngram_text, glossary

### Data

In [ ]:
PATH = "../Errors generator"
df_duo = read_csv(os.path.join(PATH, "valid_wrong_data1.csv"))

In [ ]:
ngram_text, glossary = create_ngrams_from_text(df_duo, 2)

In [ ]:
data = [". I hate that feeling where you ffeel so "]

### Results

In [ ]:
start = time.time()
spellchecker(data, ngram_text[:10000], glossary)
stop = time.time()
print((stop - start))

In [ ]:
spellchecker_1(data, v, m)

In [ ]:
list(ngrams("alahas as cat", 3))

In [ ]:
#assumptions:
#data is a list, data is given in phrases form - each element is a phrase
#corpus is a dict, corpus is given in golden_phrases : set_of_miss_spelled_phrases form
#p_corpus is a dict, is given in tokenized_golden_token: set_of_empirical_spelled_tokenized_phrases

#X - word in the window, 
#Y - observed word in the pre-window
#Z - observed word in the window
#P = (X|YZ)
# interpretation: observing the string, what expression most likely ends it


def spellchecker_2(data, gold_text, noizy_text, tokenized=False):
    if not tokenized:
        g_tokenized_text = word_tokenize(gold_text)
        n_tokenized_text = word_tokenize(noizy_text)
    else:
        g_tokenized_text = gold_text
        n_tokenized_text = noizy_text
        
    print(len(g_tokenized_text), len(n_tokenized_text))
        
    tokenized_text = zip(g_tokenized_text, n_tokenized_text)
    ngram_text = list(ngrams(tokenized_text, 2))
    correct_data = []
    pprint(list(ngram_text)[449])
    glossary = list(set(g_tokenized_text))
    
    for phrase in data:
        tokenized_phrase = ngrams(word_tokenize(phrase), 2)
        correct_phrase = []
        for token1, token2 in tokenized_phrase:
            print(token1, token2)
            
            #teraz dla tych dwoch tokenow musze znalexc token maksymalizujacy prawdopodobienstwo p = p(t2| t1, c) * p(c| t1)
            #p_1 = p(t2| t1, c), mialo byc t1c, zapisano t1t2
            #p_2 = p(c| t1), zaobserwowano t1, jakie jest prawdopodobienstwo ze nastepnie bedzie c
            #albo p=p(t1,t2|c) * p(c)
            p_max = 0
            c_max = ""
            for c in glossary:
                sub_grams = [gram for gram in ngram_text if gram[1][0] == c]
                if sub_grams:
#                     print(sub_grams)
                    p_1 = len([gram for gram in sub_grams if gram[0][1] == token1 and gram[1][1] == token2]) / len(sub_grams)
                    p_2 = len(sub_grams)
#                     print(p_1 * p_2)
                    if p_1 * p_2 > p_max:
                        p_max = p_1 * p_2
                        c_max = c
            correct_phrase.append(c_max)
            
        correct_data.append(" ".join(correct_phrase))
        
    return correct_data


In [ ]:
a = sorted([0,2,4,2])

In [ ]:
a